<a href="https://colab.research.google.com/github/0xrocky/master-thesis/blob/main/paper_training_module_extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Generic imports
#!pip install tensorflow
!pip install sage
from sage import *
import numpy as np
from google.colab import drive
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
# Neural network imports
import tensorflow as tf
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import *
import chardet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Macro
SIZE_SIMULATION_TRACES = 1 # number of synthetic traces
TAG_ADD = 1
TAG_SUB = 2
TAG_MUL = 3
TAG_END = 0 # tag of a delimitator of a modular operation

SUB_TIME = 153 # time duration of a single sub
ADD_TIME = SUB_TIME # time duration of a single sub
MUL_TIME = 2648 # time duration of a single multiplication

MAX_SUB = 255
MAX_MUL = 2804

MODULO_SUB = MAX_SUB - SUB_TIME # time duration of a modulo
MODULO_MUL = MAX_MUL - MUL_TIME # time duration of a modulo

In [ ]:
# Load from the GDrive folder the mean patterns
LOCAL_DRIVE = '/content/drive'
drive.mount( LOCAL_DRIVE, force_remount=True )

sub_of_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_sub_of_pattern.npy', allow_pickle=True )
sub_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_sub_pattern.npy', allow_pickle=True )
add_of_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_add_of_pattern.npy', allow_pickle=True )
add_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_add_pattern.npy', allow_pickle=True )
mul_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_mul_pattern.npy', allow_pickle=True )
sqr_pattern = np.load( '/content/drive/MyDrive/ECCML/MyTrim/Patterns/mean_sqr_pattern.npy', allow_pickle=True )

print( "SUB overflow mean pattern (min, max, mean, len):", np.min( sub_of_pattern ), np.max( sub_of_pattern ), np.mean( sub_of_pattern, axis=0 ), len( sub_of_pattern ) )
print( "SUB mean pattern (min, max, mean, len)         :", np.min( sub_pattern ), np.max( sub_pattern ), np.mean( sub_pattern, axis=0 ), len( sub_pattern ) )
print( "ADD overflow mean pattern (min, max, mean, len):", np.min( add_of_pattern ), np.max( add_of_pattern ), np.mean( add_of_pattern, axis=0 ), len( add_of_pattern ) )
print( "ADD mean pattern (min, max, mean, len)         :", np.min( add_pattern ), np.max( add_pattern ), np.mean( add_pattern, axis=0 ), len( add_pattern ) )
print( "MUL mean pattern (min, max, mean, len)         :", np.min( mul_pattern ), np.max( mul_pattern ), np.mean( mul_pattern, axis=0 ), len( mul_pattern ) )
print( "SQR mean pattern  (min, max, mean, len)        :", np.min( sqr_pattern ), np.max( sqr_pattern ), np.mean( sqr_pattern, axis=0 ), len( sqr_pattern ) )

Mounted at /content/drive
SUB overflow mean pattern (min, max, mean, len): -12.739841762086513 11.50184378975827 0.8494655483788998 184
SUB mean pattern (min, max, mean, len)         : -12.730101781280176 11.350560745447648 0.8084656791741394 120
ADD overflow mean pattern (min, max, mean, len): -13.274613619988617 11.053875072279535 0.9736719238216869 186
ADD mean pattern (min, max, mean, len)         : -18.16480201504563 14.576766278672247 0.8835059338571355 151
MUL mean pattern (min, max, mean, len)         : -15.8601337890625 13.431892578125 0.7690925591180787 1490
SQR mean pattern  (min, max, mean, len)        : -15.6725556640625 13.434483398437498 0.7824533051679491 1496


In [ ]:
# I choose the pattern for SHORT and LONG operations
# For SO I will use the mean pattern of adds, and for the modular SO I will use the pattern of SO plus the pattern of overflow adds
SO_TIME = SUB_TIME
LO_TIME = MUL_TIME
SO_pattern = np.concatenate( ( add_pattern, np.flip( sub_pattern ) ), axis=0 )[ : SO_TIME ]
SO_pattern_modulo = np.concatenate( ( SO_pattern, np.flip( add_of_pattern ) ), axis=0 )[ : SO_TIME + MODULO_SUB ]
LO_pattern = np.concatenate( ( sqr_pattern, mul_pattern ), axis=0 )[ : LO_TIME ]
LO_pattern_modulo = np.concatenate( ( LO_pattern, np.flip( mul_pattern ) ), axis=0 )[ : LO_TIME + MODULO_MUL ]

print( len( SO_pattern ) )
print( len( LO_pattern ) )
print( len( SO_pattern_modulo ) )
print( len( LO_pattern_modulo ) )

153
2648
255
2804


In [ ]:
# SAVEZ
# Da altro Notebook
arr = np.load('/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/Traces/trace_single_Tue_Oct__4_22:15:13_2022.npz', allow_pickle=True, mmap_mode='r' )

#with open( '/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/trace_single.npz', 'rb' ) as f:
#  result = chardet.detect( f.read() )
#arr = np.load('/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/trace_single.npz', allow_pickle=True, mmap_mode='r', encoding=result['encoding'] )
#arr = np.load('/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/trace_single.npz', allow_pickle=True, mmap_mode='r' )
traces = arr['x']
#parameters = arr['y']

# SAVE
#arr = np.load('/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/traces.npy', allow_pickle=True )
#arr = np.load('/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/trace_single.npy', allow_pickle=True )
print( "ndim: ", np.ndim( arr ) )
print( "shape: ", np.shape( arr ) )
print( "size: ", np.size( arr ) )
print( "len: ", len( arr ) )
#print( "len: ", len( arr['x'] ) )
print( "ndim: ", np.ndim( traces ) )
print( "shape: ", np.shape( traces ) )
print( "size: ", np.size( traces ) )
print( "len: ", len( traces ) )
print( traces[ 0 ][ 0 ] )
#print( len( traces[1] ) )
#print( len( parameters ) )

ndim:  1
shape:  (3,)
size:  3
len:  3
ndim:  2
shape:  (6873726, 2)
size:  13747452
len:  6873726
1


In [ ]:
# Per mia comodità costruisco delle funzioni per ottenere un sample, un tag, i cicli di clock
def get_trace_hw( trace, i ):
  return trace[ i ][ 0 ]

def get_trace_tag( trace, i ):
  return trace[ i ][ 1 ]

In [ ]:
# adding the pattern for that operation
def hamming_weight_with_pattern( hw_op, pattern_op ):
  temp = [ x[ 0 ] for x in hw_op ]
  return np.add( pattern_op, temp )

In [ ]:
# Adding Gaussian vertical noise at the hamming weight values with pattern
mean = 0
var = 1.5

def ver_noise_pattern_hamming_weight( samples ):
  return [ sample + np.random.normal( loc=mean, scale=var ) for sample in samples ]

In [ ]:
# Generate a max duration for a specific operation with a plus horizontal noise of 10%
def hor_noise( operation_time ):
  return operation_time + round( operation_time * 10 / 100 )

MUL_TIME_hnoise = hor_noise( MUL_TIME )
SUB_TIME_hnoise = hor_noise( SUB_TIME )

In [ ]:
# Populating a trace with its mul and add/sub operations
def traces_populating( trace, op_time_hnoise, modular_operation, pattern_operation, tag ):
  temp = []
  hamming_weight_pattern_vnoise = ver_noise_pattern_hamming_weight( hamming_weight_with_pattern( modular_operation, pattern_operation ) )
  # op_duration_horizontal_noise = random.randint( op_time, op_time_hnoise )
  # For each sample of operation: trace = [ [ ( hw1, tag1 ), ( hw1, tag1 ), ... ], [ ( hw2, tag2 ), ( hw2, tag2 ), ... ], ... ]
  # for j in range( op_duration_horizontal_noise ):
  #  temp += [ ( hamming_weight_pattern_vnoise[ j ], tag ) ]
  for hw in hamming_weight_pattern_vnoise:
    temp += [ ( hw, tag ) ]
  trace += [ temp ]

In [ ]:
# Conto le ADD, SUB e MUL
for trace in range( SIZE_SIMULATION_TRACES ):
  adds = 0
  subs = 0
  muls = 0
  for couple in range( len( traces ) ):
    hw = get_trace_hw( traces, couple )
    tag = get_trace_tag( traces, couple )
    if( tag == TAG_END ):
      tag_prec = get_trace_tag( traces, couple - 1 )
      if tag_prec == TAG_ADD:
        adds += 1
      elif tag_prec == TAG_SUB:
        subs += 1
      elif tag_prec == TAG_MUL:
        muls += 1
  print( "adds", adds )
  print( "subs", subs )
  print( "muls", muls )
  print( "TOT", adds+subs+muls )

adds 322
subs 2549
muls 2242
TOT 5113


In [ ]:
# Tiro via il delimitatore TAG_END che mi serviva per segnalare la fine di un'operazione modulare
final_traces = []
cont = 0

for trace in range( SIZE_SIMULATION_TRACES ):
  final_trace = []
  temp = []
  for couple in range( len( traces ) ):
    hw = get_trace_hw( traces, couple )
    tag = get_trace_tag( traces, couple )
    if tag != TAG_END:
      cont += 1
      temp += [ ( hw, tag ) ]
    else:
      final_trace += [ temp ]
      temp = []
      #print( "tag", get_trace_tag( traces, couple-1 ), cont )
      cont = 0
  final_traces += [ final_trace ]

In [ ]:
print( len ( final_traces ) )
print( len ( final_traces[ 0 ] ) )
print( final_traces[ 0 ][ :2 ] )

1
5113
[[(1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3),

In [ ]:
# Appendin the boolean TAG True/False for each sample
synthetic_traces = []
cont_true = 0
cont_false = 0

for trace in final_traces:
  for modular_operation in trace:
    duration = len( modular_operation )
    # ADD / SUB case
    if get_trace_tag( modular_operation, 0 ) == TAG_ADD or get_trace_tag( modular_operation, 0 ) == TAG_SUB:
      # modulo?
      cont_true += duration
      if( duration > SO_TIME ):
        traces_populating( synthetic_traces, None, modular_operation, SO_pattern_modulo, True )
      else:
        traces_populating( synthetic_traces, None, modular_operation, SO_pattern, True )
    # MUL case
    elif get_trace_tag( modular_operation, 0 ) == TAG_MUL:
      if cont_false > 582981:
        continue
      else:
        cont_false += duration
        # modulo?
        if( duration > LO_TIME ):
          traces_populating( synthetic_traces, None, modular_operation, LO_pattern_modulo, False )
        else:
          traces_populating( synthetic_traces, None, modular_operation, LO_pattern, False )
  #random.shuffle( synthetic_traces )
print( "Samples ADD/SUB", cont_true )
print( "Samples MUL", cont_false )

Samples ADD/SUB 582981
Samples MUL 585100


In [ ]:
print( len( synthetic_traces ) )
print( synthetic_traces[ :4 ] )

3080
[[(9.498235795206329, False), (2.508146332039656, False), (-2.6813537207733393, False), (-0.6948063289502009, False), (3.736676458237858, False), (9.963407837757913, False), (3.085251619111523, False), (6.628701872549075, False), (2.141997917607105, False), (5.026298455953928, False), (-1.6086103430586252, False), (-4.483686809093781, False), (6.94949727359998, False), (6.456760889275071, False), (-1.7259303513654993, False), (-1.3990763400225044, False), (13.080858898658194, False), (2.1988096855902017, False), (0.40031406764958477, False), (11.688440295900444, False), (-2.762906253188661, False), (2.0378171316735454, False), (3.4581605321371853, False), (-4.189969874173354, False), (3.7917892158650313, False), (3.9291609737356263, False), (-12.912548297048886, False), (8.533953173129607, False), (-1.8999898533138122, False), (6.244269565823348, False), (3.6665062005883096, False), (-3.2320467220052773, False), (9.157856736714198, False), (-3.4336255464107994, False), (1.47103707

In [ ]:
final_traces = []
weights_trace = []
tags_trace = []

for trace in range( SIZE_SIMULATION_TRACES ):
  temp_trace_hw = []
  temp_trace_tag = []
  for modular_operation in synthetic_traces:
    for couple_hw_tag in modular_operation:
      #weights_traces += [ couple_hw_tag[ 0 ] ]
      #tags_traces += [ couple_hw_tag[ 1 ] ]
      temp_trace_hw += [ couple_hw_tag[ 0 ] ]
      temp_trace_tag += [ couple_hw_tag[ 1 ] ]
  weights_trace += [ temp_trace_hw ]
  tags_trace += [ temp_trace_tag ]

size_weights_simulation_trace = len( weights_trace )
size_tags_simulation_trace = len( tags_trace )
size_weights_single_trace = len( weights_trace[ 0 ] )
size_tags_single_trace = len( tags_trace[ 0 ] )

print( synthetic_traces[ 0 ][:100] )
print( weights_trace[ 0 ][:100] )
print( tags_trace[ 0][:100] )

print( "Size of the list of all simulation traces (numbers of executions):", SIZE_SIMULATION_TRACES )
print( "Size of the list of all weights traces constructed from the simulation traces:", size_weights_simulation_trace )
print( "Size of the list of all tags traces constructed from the simulation traces:", size_tags_simulation_trace )
#print( "Size of the only first simulation trace", SIZE_SINGLE_TRACE )
print( "Size of the only first weight trace", size_weights_single_trace )
print( "Size of the only first tags trace", size_tags_single_trace )

if SIZE_SIMULATION_TRACES != size_weights_simulation_trace or SIZE_SIMULATION_TRACES != size_tags_simulation_trace:
  print( "ERROR: number of total weights not equal at number of total tags in the simulation traces" )
#if size_tags_single_trace != size_weights_single_trace:  print( "ERROR: number of weights of the first trace not equal at number of tags of the first trace" )

[(9.498235795206329, False), (2.508146332039656, False), (-2.6813537207733393, False), (-0.6948063289502009, False), (3.736676458237858, False), (9.963407837757913, False), (3.085251619111523, False), (6.628701872549075, False), (2.141997917607105, False), (5.026298455953928, False), (-1.6086103430586252, False), (-4.483686809093781, False), (6.94949727359998, False), (6.456760889275071, False), (-1.7259303513654993, False), (-1.3990763400225044, False), (13.080858898658194, False), (2.1988096855902017, False), (0.40031406764958477, False), (11.688440295900444, False), (-2.762906253188661, False), (2.0378171316735454, False), (3.4581605321371853, False), (-4.189969874173354, False), (3.7917892158650313, False), (3.9291609737356263, False), (-12.912548297048886, False), (8.533953173129607, False), (-1.8999898533138122, False), (6.244269565823348, False), (3.6665062005883096, False), (-3.2320467220052773, False), (9.157856736714198, False), (-3.4336255464107994, False), (1.47103707307509

In [ ]:
yes = no = 0
for i in range( size_tags_single_trace ):
  if tags_trace[ 0 ][ i ] == True:
    yes += 1
  else:
    no += 1
print( "tag True", yes )
print( "tag False", no )

tag True 582981
tag False 585100


In [ ]:
# Sliding window to create the dataset: fixing a window size and an advancement step, the aim is focusing on an Hamming weight window and tag
# with a 1 the window contains a TRUE tag, 0 otherwise
window = 500
step = 10
X_full_dataset = []
Y_full_dataset = []

for trace in range( SIZE_SIMULATION_TRACES ):
  for i in range( 0, size_weights_single_trace - window + 1, step ):
    X_full_dataset += [ weights_trace[ trace ][ i:i + window ] ]
    window_of_tags = np.array( tags_trace[ trace ][ i:i + window ] )
    set_True = 0
    # It detects where chunks begin, has some logic for the first and last chunk, and simply computes differences between chunk starts and discards lengths corresponding to False chunks
    arr = np.diff( np.where( np.concatenate( ( [ window_of_tags[ 0 ] ], window_of_tags[ :-1 ] != window_of_tags[ 1: ], [ True ] ) ) )[ 0 ] )[ ::2 ]
    if np.size( arr ) > 0 and np.max( arr ) >= SO_TIME:
      set_True = 1
    Y_full_dataset += [ set_True ]

size_X_full_dataset = len( X_full_dataset )
size_Y_full_dataset = len( Y_full_dataset )
size_single_trace_X = len( X_full_dataset[ 0 ] )

print( X_full_dataset[ :10 ] )
print( Y_full_dataset[ :100 ] )
print( "Size of the X dataset of {} sliding window of Hamming weights:".format( window ), size_X_full_dataset )
print( "Size of the Y dataset of {} sliding window of tags:".format( window ), size_Y_full_dataset )
print( "Size of the first trace of the X dataset of Hamming weight in sliding window:", size_single_trace_X )

[[9.498235795206329, 2.508146332039656, -2.6813537207733393, -0.6948063289502009, 3.736676458237858, 9.963407837757913, 3.085251619111523, 6.628701872549075, 2.141997917607105, 5.026298455953928, -1.6086103430586252, -4.483686809093781, 6.94949727359998, 6.456760889275071, -1.7259303513654993, -1.3990763400225044, 13.080858898658194, 2.1988096855902017, 0.40031406764958477, 11.688440295900444, -2.762906253188661, 2.0378171316735454, 3.4581605321371853, -4.189969874173354, 3.7917892158650313, 3.9291609737356263, -12.912548297048886, 8.533953173129607, -1.8999898533138122, 6.244269565823348, 3.6665062005883096, -3.2320467220052773, 9.157856736714198, -3.4336255464107994, 1.4710370730750957, 4.423393421727251, 2.0183655505050497, -4.529173151100222, 0.39735795853645595, 5.746601723359317, 5.36732808499378, 6.7303174625080056, 0.5715249076073436, -0.2854451328040435, 1.4941859945550526, 6.586054426087135, -5.009702200269032, 7.086367553914385, 0.7937271738082103, 5.394259535909319, 1.35190

In [ ]:
yes = 0
no = 0
for i in range( size_Y_full_dataset ):
  if Y_full_dataset[ i ] == 1:
    yes += 1
  else:
    no += 1
print( "finestre taggate a 1", yes )
print( "finestre taggate a 0", no )

finestre taggate a 1 61036
finestre taggate a 0 55723


In [ ]:
X_full_dataset = np.array( X_full_dataset )
X_full_dataset = np.reshape( X_full_dataset, ( X_full_dataset.shape[ 0 ], X_full_dataset.shape[ 1 ], 1 ) )
Y_full_dataset = to_categorical( np.array( Y_full_dataset ) )
print( "Shapes:" )
print( "\tX_full_dataset" + str( X_full_dataset.shape ) )
print( "\tY_full_dataset" + str( Y_full_dataset.shape ) )

Shapes:
	X_full_dataset(116759, 500, 1)
	Y_full_dataset(116759, 2)


In [ ]:
# Building the LSTM
model = Sequential()
model.add( Conv1D( filters=64, kernel_size=3, activation='relu', input_shape=( X_full_dataset.shape[ 1 ], X_full_dataset.shape[ 2 ] ) ) )
model.add( AveragePooling1D( pool_size=10 ) )
model.add( LSTM( 1000 ) )
model.add( Dropout( 0.5 ) )
model.add( Dense( 1000, activation='relu' ) )
model.add( Dense( 2, activation='softmax' ) )
model.compile( loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'] )
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 498, 64)           256       
                                                                 
 average_pooling1d (AverageP  (None, 49, 64)           0         
 ooling1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 1000)              4260000   
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 2)                 2002      
                                                        

In [ ]:
model.fit( X_full_dataset, Y_full_dataset, epochs=100, batch_size=64, validation_split=0.20 )

Epoch 1/100
1460/1460 [==============================] - 59s 31ms/step - loss: 0.6817 - accuracy: 0.5805 - val_loss: 0.9477 - val_accuracy: 2.1411e-04
Epoch 2/100
1460/1460 [==============================] - 47s 32ms/step - loss: 0.6720 - accuracy: 0.5874 - val_loss: 0.9411 - val_accuracy: 0.0000e+00
Epoch 3/100
1460/1460 [==============================] - 48s 33ms/step - loss: 0.6703 - accuracy: 0.5886 - val_loss: 0.9453 - val_accuracy: 0.0000e+00
Epoch 4/100
1460/1460 [==============================] - 49s 33ms/step - loss: 0.6688 - accuracy: 0.5906 - val_loss: 0.9331 - val_accuracy: 0.0000e+00
Epoch 5/100
1460/1460 [==============================] - 49s 33ms/step - loss: 0.6673 - accuracy: 0.5909 - val_loss: 1.0727 - val_accuracy: 0.0000e+00
Epoch 6/100
1460/1460 [==============================] - 49s 33ms/step - loss: 0.6667 - accuracy: 0.5914 - val_loss: 1.0499 - val_accuracy: 0.0000e+00
Epoch 7/100
1460/1460 [==============================] - 49s 34ms/step - loss: 0.6652 - accura

In [ ]:
# Save model
LOCAL_DRIVE = '/content/drive'
drive.mount( LOCAL_DRIVE, force_remount=True )

filepath = "/content/drive/MyDrive/Colab Notebooks/[SEC_PAT]Shared/Paper/Models_trained/Model_Trace_ext/model"
print("Saving model...")
model.save( filepath, save_format="h5" )
print("Done.")

Mounted at /content/drive
Saving model...
Done.
